# Other Groupby Methods

There are many other groupby methods than `agg`, `filter`, and `transform`. In this chapter, you'll learn how to discover and use them.

## Kinds of groupby attributes and methods

All groupby methods act on either a Series or a DataFrame. If there is a single column name within the brackets following the call to the `groupby` method, then it acts on a Series. If there are no brackets or multiple column names in the brackets, then then it acts on a DataFrame. Let's see some examples of the two kinds of `groupby` methods available. Let's begin by reading in the San Francisco employee compensation dataset.

In [ ]:
import pandas as pd
import numpy as np
sf_emp = pd.read_csv('../data/sf_employee_compensation.csv')
sf_emp.head(3)

All grouping begins with a call to the `groupby` method, providing it the grouping column(s). Let's assign the object returned when grouping by organization group and output its type.

In [ ]:
g_df = sf_emp.groupby('organization group')
type(g_df)

The technical name for this object is `DataFrameGroupBy` and its methods can act on the entire DataFrame. Let's call the same `groupby` method, but this time put the salaries column in brackets following it. A `SeriesGroupBy` is produced and its methods can only act on the salaries Series.

In [ ]:
g_series = sf_emp.groupby('organization group')['salaries']
type(g_series)

### Explore the differences between the objects with the `agg` method

Let's use the `agg` method to explore the differences between the `g_df` and `g_series` objects. With `g_df`, you can aggregate any of the other columns, not just salaries. Here, we find the mean salary and max retirement for each organization group.

In [ ]:
g_df.agg(mean_salary=('salaries', 'mean'), max_retirement=('retirement', 'max')) \
    .round(-3).style.format('{:,.0f}')

With `g_series`, you only have access to the salaries column and not any other column.

In [ ]:
g_series.agg(mean_salary=('salaries', 'mean')).round(-3).style.format('{:,.0f}')

### More aggregations with each object

Both `g_df` and `g_series` allow you to aggregate with multiple different syntaxes. For instance, you can pass in a list of aggregation strings to the `agg` method. For `g_df`, this will perform the aggregation on all of the non-grouping columns (and silently drop the columns where the operation fails). A multi-level column index will be returned.

In [ ]:
g_df.agg(['mean', 'median']).round(-3).style.format('{:,.0f}')

With `g_series`, only the salaries column will be aggregated.

In [ ]:
g_series.agg(['mean', 'median']).round(-3).style.format('{:,.0f}')

Hopefully, the difference between DataFrameGroupBy and SeriesGroupBy objects are clear. The former has the entire DataFrame available to be acted on, while the latter only acts on the single Series selected after the call to `groupby`.

## Finding all available attributes and methods

The vast majority of DataFrameGroupBy and SeriesGroupBy attributes and methods overlap. Let's retrieve all the public attributes and methods for each and print out the SeriesGroupBy ones to the screen.

In [ ]:
public_gbs_methods = [m for m in dir(g_series) if not m.startswith('_')]
public_gbdf_methods = [m for m in dir(g_df) if not m.startswith('_') 
                       and m not in sf_emp.columns]
for i, method in enumerate(public_gbs_methods):
    end = '\n' if i % 4 == 3 else ''
    print(f'{method:25}', end=end)

You should be familiar with many of these attributes and methods as they overlap with the ones available directly from a normal Series. We can take the set difference to determine the attributes and methods unique to each one. These are unique to SeriesGroupBy.

In [ ]:
set(public_gbs_methods) - set(public_gbdf_methods)

These are unique to DataFrameGroupBy objects.

In [ ]:
set(public_gbdf_methods) - set(public_gbs_methods)

## Calling single aggregation methods

You can bypass the `agg` method by calling the aggregation method directly from one of the groupby objects. The disadvantage is that you won't be able to rename the resulting column. Here, we take the maximum of the salaries column for each organization group.

In [ ]:
g_series.max()

With `g_df`, the maximum of all non-grouping columns is returned. Notice that the job column shows up in the result. This is because the `max` method works for strings in an object data type column.

In [ ]:
g_df.max()

### The entire syntax

It's rare that the intermediate call to the `groupby` method will be assigned to a variable as we've done in this chapter. We are only doing this to avoid the repetitive nature of calling the same method over again. When completing these operations in practice, you'll likely begin with the original DataFrame, call the `groupby` method, and then chain the grouping method you desire. Below, the full syntax is given for the last two operations.

```python
sf_emp.groupby('organization group')['salaries'].max()
sf_emp.groupby('organization group').max()
```

### More aggregating methods

Most of the aggregating methods available to normal Series and DataFrames are available to their groupby counterparts. Nearly all of them return a single value for each group. However, the `describe` method returns many aggregations. You can provide it a list of percentiles to return as well. Here, we get many summary statistics for the salaries column on all the groups.

In [ ]:
g_series.describe(percentiles=[.01, .2, .5, .8, .99]).round(0).style.format('{:,.0f}')

Calling the `describe` method on `g_df` would return a very wide DataFrame with all of these statistics calculated on each numeric column.

## `head`, `tail`, and `nth` groupby methods

The `head` and `tail` methods return the first and last five rows, respectively, of each group. Set the parameter `n` to an integer to control the number of rows returned per group. Here we return the first two rows of the entire DataFrame for each organization group. Notice that the order of the rows are preserved and they are not sorted by the grouping column.

In [ ]:
g_df.head(2)

Using the same operation on a `g_series` isn't as clear as only the values of the salaries are returned without the context of the grouping column.

In [ ]:
g_series.head(2)

The `nth` groupby method allows you to control exactly which rows from the group are returned using integer location. Pass it a single integer or a list of integers. For instance, the following returns rows with integer location 5 and 10 from each group.

In [ ]:
g_df.nth([5, 10])

## Non-aggregating methods

Most of the other methods do not aggregate and instead return a Series or DataFrame with the same length as the group. For the most part, they work exactly the same as they do on regular Series or DataFrames. To help teach these methods, a small fake DataFrame will be created.

In [ ]:
df = pd.DataFrame({'item': ['A', 'B', 'A', 'A', 'B', 'A', 'B', 'B'],
                   'quantity': [5, 3, 8, np.nan, 2, 15, np.nan, 6]})
df

We'll use a SeriesGroupBy object to showcase these methods. 

In [ ]:
g_series = df.groupby('item')['quantity']

All of the methods in this section preserve the order of the original values. They do NOT sort by the group. Take for instance, the `cumsum` method which accumulates the sum beginning from the top by group.

In [ ]:
g_series.cumsum()

A Series is returned, but is difficult to decipher without it being attached to the original DataFrame. Let's add it as a column and then re-examine the output.

In [ ]:
df['quantity_cumsum'] = g_series.cumsum()
df

Each group has the quantity column accumulated independently for each group. The method `cumcount` is unique to groupby objects and provides the integer location of each row by group beginning with 0.

In [ ]:
df['group_iloc'] = g_series.cumcount()
df

Each quantity can be ranked using the `rank` method. Below, the largest quantity of each group gets ranked 1.

In [ ]:
df['group_rank'] = g_series.rank(ascending=False)
df

We fill in missing values with the previous know missing value of that group with the `fillna` method.

In [ ]:
df['group_ffill'] = g_series.fillna(method='ffill')
df

## Exercises

Execute the next cell to read in some of the columns from the flights dataset and use it to answer the following exercises.

In [ ]:
import pandas as pd
cols = ['date', 'airline', 'origin', 'dest', 'dep_time', 'arr_time',
       'cancelled', 'air_time', 'distance', 'carrier_delay']
flights = pd.read_csv('../data/flights.csv', parse_dates=['date'], usecols=cols)
flights.head(3)

### Exercise 1

<span style="color:green; font-size:16px">For each airline, return the first and last row of each group. Use the `nth` group by method.</span>

### Exercise 2

<span style="color:green; font-size:16px">For every origin and destination combination, select the 5000th flight.</span>

### Exercise 3

<span style="color:green; font-size:16px">Find the date of the 10th cancelled flight for each airline.</span>

### Exercise 4

<span style="color:green; font-size:16px">Find the average carrier delay for each origin and destination combination with more than 300 flights.</span>